### Below is the main conditioning program. 

#### Steps:

1. Get VIIRS bounding boxes

2. For each MERRA datapoint:
    1. Get distance from closest VIIRS bounding box - throw out ones that are too far (dist > 40)
    2. Store value of datapoint
    3. getROI of datapoint
    4. gridVIIRS of datapoint - if it's too sparse (i.e. num nonzero > 80%)
    5. Save gridded VIIRS data, metadata of MERRA datapoint
    

In [1]:
import os
import tarfile
import xarray as xr
import numpy as np
import pickle
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import matplotlib.path as paths
import cartopy.crs as ccrs

In [5]:
## Extracts boundary metadata from a particular netcdf file
def extractBounds(filename):
    dataset = xr.open_dataset(filename)
    pp = dataset.geospatial_bounds[9:-2].split(",")
    polygon = []
    center = np.array([0, 0])
    for point in pp:
        pt = np.array(list(map(float, point.split(" "))))
        polygon.append(pt)
        center = center + pt
    np_poly = np.array(polygon)
    
    return {"bounds": np_poly, "btime": dataset.time_coverage_start,
            "etime": dataset.time_coverage_end, "center": center / len(polygon)}

## Squishes the larger tarfiles to make things faster.
def extractNestedTars(folder):
    tar_files = os.listdir(folder)
    for tar_f in tar_files:
        if (tar_f == "metadata" or tar_f == "merra.nc"):
            continue
        print("Extracting " + tar_f)
        tar = tarfile.open(folder + tar_f)
        tar.extractall(path= folder + "extracted/")
        tar.close()

## Gets the boundary metadata of all VIIRS files within a directory
def getVIIRSMetadata(folder):
    metadata = {}
    tar_files = os.listdir(folder)
    
    for tar_f in tar_files:
            
        if (tar_f == "metadata" or tar_f == "merra.nc"):
            continue
        print("Extracting " + tar_f)
        tar = tarfile.open(folder + "/" + tar_f)
        tar.extractall(path="data/curr_data")
        tar.close()
            
        files = os.listdir("data/curr_data")
        for f in files:
            print("Opening " + f)
            
            ## THIS IS WHERE YOU CAN APPLY SOME SORT OF PROCESSING STEP TO ALL THE DATA
            metadata[tar_f + "/" + f] = extractBounds("data/curr_data/" + f)
        
            print("Deleting " + f)
            os.remove("data/curr_data/" + f)
        print("Done extracting " + tar_f)
        
    print("Done extracting")

    with open(folder + "metadata", "wb") as handle:
        pickle.dump(metadata, handle)
        
    return metadata

## Gets the VIIRS file closest to a given point
def getClosestVIIRSSwath(point, metadata):
        m_files = np.array(list(metadata.keys()))
        m_vals = np.array(list(metadata.values()))
        c_points = []
        for vals in m_vals:
            c_points.append(vals["center"])
        # Gets closest point
        c_array = np.asarray(c_points)
        dists = np.sum((c_array - point)**2, axis=1)
        return [m_files[np.argmin(dists)], np.min(dists)]

## Extracts and opens the VIIRS data corresponding to the full filename/path
def getVIIRSData(folder, filename):
    levels = filename.split("/")
    
    print("Extracting " + levels[0])
    tar = tarfile.open(folder + levels[0])
    tar.extractall(path="data/curr_data")
    tar.close()
    
    print("Reading " + levels[1])
    data = xr.open_dataset("data/curr_data/" + levels[1])
    
    for f in os.listdir("data/curr_data/"):
        os.remove("data/curr_data/" + f)
    
    return data

## Converts VIIRS data to an array for linear interpolation.
def getVIIRSArrays(viirs_data):
    sample_points = []
    aod_array = []
    
    aods = viirs_data.AOD550
    aod_array = aods.values.reshape(1, -1)[0]
    lons = aods.Longitude.values.reshape(1, -1)[0]
    lats = aods.Latitude.values.reshape(1, -1)[0]
    sample_points = np.column_stack((lons, lats))
    return [sample_points, aod_array]

## Creates a ROI for the particular MERRA datapoint
def getMERRAROI(lon, lat):
    d_lon = 0.625
    d_lat = 0.5
    polygon = []
    bounds = {"minLon": lon - d_lon, "maxLon": lon + d_lon,
              "minLat": lat - d_lat, "maxLat": lat + d_lat}
    polygon.append(np.array([bounds["minLon"], bounds["minLat"]]))
    polygon.append(np.array([bounds["minLon"], bounds["maxLat"]]))
    polygon.append(np.array([bounds["maxLon"], bounds["maxLat"]]))
    polygon.append(np.array([bounds["maxLon"], bounds["minLat"]]))
    polygon.append(np.array([bounds["minLon"], bounds["minLat"]]))
    return [np.array(polygon), bounds]

        
def inSwath(roi, swath_polygon):
    path = paths.Path(np.array(swath_polygon))
    for point in roi:
        if not path.contains_point(point):
            return False
    return True

## Performs nearest neighbor linear interpolation on the VIIRS data array.
def VIIRStoGrid(s_points, data_array, boundary):
    numLonPoints = 100j
    numLatPoints = 80j
    
    lon_grid, lat_grid = np.mgrid[boundary["minLon"]:boundary["maxLon"]:numLonPoints, 
                                  boundary["minLat"]:boundary["maxLat"]:numLatPoints]
    
    gridded_data = griddata(s_points, data_array, (lon_grid, lat_grid), method="nearest")
    return gridded_data

def getT(m_data, swath_name):
    time = list(map(int, m_data[swath_name]["btime"].split("T")[1][:-1].split(":")))
    if time[1] > 30:
        return time[0] + 1
    else:
        return time[0]

In [6]:
def process(folder):
    metadata = None
    if "metadata" in os.listdir(folder):
        with open(folder + "metadata", "rb") as md_file:
            metadata = pickle.load(md_file)
    else:
        metadata = getVIIRSMetadata(folder)
        
    merra_data = xr.open_dataset(folder + "merra.nc")
    
    prev_swath = ""
    viirs_data = None
    viirs_arrays = None
    for y in range(100, len(merra_data.lat) - 100):
        for x in range(60, len(merra_data.lon) - 60):
            # Getting lat/lon of merra gridpoint
            point = [merra_data.lon.values[x], merra_data.lat.values[y]]
            print("\nConsidering merra point at " + str(point) + "\nIndex " + str(x) + ", " + str(y))
            
            # Getting corresponding VIIRs box
            [closest_viirs, distance] = getClosestVIIRSSwath(point, metadata)
            
            # Skipping point if too far from center of swath
            print(distance)
            if (distance > 44):
                print("Too far from center of VIIRS data.")
                continue
                
            
            # Gets the size of the box around the MERRA point
            print("Getting ROI")
            roi = getMERRAROI(point[0], point[1])
            #print("Closest VIIR swath: " + closest_viirs)
            
            # Selecting correct time for observation
            time = getT(metadata, closest_viirs)
            merra_val = merra_data.TOTEXTTAU.values[time][y][x]
            print("MERRA value: " + str(merra_val) + " taken at time " + str(time) + "hrs")
            
            
            if not inSwath(roi[0][:-1], metadata[closest_viirs]["bounds"][:-1]):
                print("Not in satellite swath, so discarding")
                continue
            
            # Updating satellite data if the swath has changed.
            if (prev_swath != closest_viirs):
                print("Retrieving VIIRS data from closest swath.")
                viirs_data = getVIIRSData(folder, closest_viirs)
                viirs_arrays = getVIIRSArrays(viirs_data)
            prev_swath = closest_viirs
            
            
            
            # Interpolates VIIRS data to 100x80 grid. This is the slowest part.
            print("Gridding viirs")
            gridded_viirs = VIIRStoGrid(viirs_arrays[0], viirs_arrays[1], roi[1])
            print("Counting number of points")
            num_points = np.count_nonzero(~np.isnan(gridded_viirs))
            print("Number of points: " + str(num_points))
            if (num_points < 100):
                print("Discarding datapoint")
                continue
                
            print("Saving data")
            # Saving the VIIRS data to a pickle file
            with open(folder + "processed/{:04d}_{:04d}".format(x, y), "wb") as datafile:
                pickle.dump({"gridded_viirs":gridded_viirs, "roi": roi,
                             "merra_val":merra_val, "closest_viirs":closest_viirs}, datafile)
                
    

In [8]:
process("data/VIIRS-20200320/")


Considering merra point at [-142.5, -40.0]
Index 60, 100
77.13669799039988
Too far from center of VIIRS data.

Considering merra point at [-141.875, -40.0]
Index 61, 100
66.9528229903999
Too far from center of VIIRS data.

Considering merra point at [-141.25, -40.0]
Index 62, 100
57.5501979903999
Too far from center of VIIRS data.

Considering merra point at [-140.625, -40.0]
Index 63, 100
48.92882299039991
Too far from center of VIIRS data.

Considering merra point at [-140.0, -40.0]
Index 64, 100
41.088697990399915
Getting ROI
MERRA value: 0.08306982 taken at time 23hrs
Not in satellite swath, so discarding

Considering merra point at [-139.375, -40.0]
Index 65, 100
34.02982299039992
Getting ROI
MERRA value: 0.07251074 taken at time 23hrs
Not in satellite swath, so discarding

Considering merra point at [-138.75, -40.0]
Index 66, 100
27.752197990399928
Getting ROI
MERRA value: 0.07571509 taken at time 23hrs
Not in satellite swath, so discarding

Considering merra point at [-138.125,

Counting number of points
Number of points: 90
Discarding datapoint

Considering merra point at [-110.625, -40.0]
Index 111, 100
9.938393385999944
Getting ROI
MERRA value: 0.13351049 taken at time 21hrs
Gridding viirs
Counting number of points
Number of points: 13
Discarding datapoint

Considering merra point at [-110.0, -40.0]
Index 112, 100
6.468693385999955
Getting ROI
MERRA value: 0.13503636 taken at time 21hrs
Gridding viirs
Counting number of points
Number of points: 1
Discarding datapoint

Considering merra point at [-109.375, -40.0]
Index 113, 100
3.7802433859999667
Getting ROI
MERRA value: 0.14004125 taken at time 21hrs
Gridding viirs
Counting number of points
Number of points: 1
Discarding datapoint

Considering merra point at [-108.75, -40.0]
Index 114, 100
1.8730433859999773
Getting ROI
MERRA value: 0.14883031 taken at time 21hrs
Gridding viirs
Counting number of points
Number of points: 0
Discarding datapoint

Considering merra point at [-108.125, -40.0]
Index 115, 100
0.7

Counting number of points
Number of points: 20
Discarding datapoint

Considering merra point at [-81.25, -40.0]
Index 158, 100
3.523834521999976
Getting ROI
MERRA value: 0.08684495 taken at time 20hrs
Gridding viirs
Counting number of points
Number of points: 15
Discarding datapoint

Considering merra point at [-80.625, -40.0]
Index 159, 100
5.877684521999967
Getting ROI
MERRA value: 0.08882859 taken at time 20hrs
Gridding viirs
Counting number of points
Number of points: 57
Discarding datapoint

Considering merra point at [-80.0, -40.0]
Index 160, 100
9.012784521999958
Getting ROI
MERRA value: 0.08483079 taken at time 20hrs
Not in satellite swath, so discarding

Considering merra point at [-79.375, -40.0]
Index 161, 100
12.929134521999949
Getting ROI
MERRA value: 0.08229783 taken at time 20hrs
Not in satellite swath, so discarding

Considering merra point at [-78.75, -40.0]
Index 162, 100
17.62673452199994
Getting ROI
MERRA value: 0.07207444 taken at time 20hrs
Not in satellite swath,

Counting number of points
Number of points: 2712
Saving data

Considering merra point at [-50.0, -40.0]
Index 208, 100
42.907073965599984
Getting ROI
MERRA value: 0.08037045 taken at time 18hrs
Gridding viirs
Counting number of points
Number of points: 2650
Saving data

Considering merra point at [-49.375, -40.0]
Index 209, 100
51.09357396559999
Too far from center of VIIRS data.

Considering merra point at [-48.75, -40.0]
Index 210, 100
60.06132396559998
Too far from center of VIIRS data.

Considering merra point at [-48.125, -40.0]
Index 211, 100
69.81032396559998
Too far from center of VIIRS data.

Considering merra point at [-47.5, -40.0]
Index 212, 100
80.34057396559997
Too far from center of VIIRS data.

Considering merra point at [-46.875, -40.0]
Index 213, 100
91.65207396559997
Too far from center of VIIRS data.

Considering merra point at [-46.25, -40.0]
Index 214, 100
103.74482396559998
Too far from center of VIIRS data.

Considering merra point at [-45.625, -40.0]
Index 215,

567.1600873552002
Too far from center of VIIRS data.

Considering merra point at [93.75, -40.0]
Index 438, 100
582.4640123552001
Too far from center of VIIRS data.

Considering merra point at [94.375, -40.0]
Index 439, 100
598.5491873552002
Too far from center of VIIRS data.

Considering merra point at [95.0, -40.0]
Index 440, 100
587.440882681999
Too far from center of VIIRS data.

Considering merra point at [95.625, -40.0]
Index 441, 100
559.986307681999
Too far from center of VIIRS data.

Considering merra point at [96.25, -40.0]
Index 442, 100
533.2433125024
Too far from center of VIIRS data.

Considering merra point at [96.875, -40.0]
Index 443, 100
505.39968750240007
Too far from center of VIIRS data.

Considering merra point at [97.5, -40.0]
Index 444, 100
478.3373125024001
Too far from center of VIIRS data.

Considering merra point at [98.125, -40.0]
Index 445, 100
452.05618750240006
Too far from center of VIIRS data.

Considering merra point at [98.75, -40.0]
Index 446, 100
42

...

Reading JRR-AOD_v2r1_j01_s202003200607558_e202003200609203_c202003200650480.nc
Gridding viirs
Counting number of points
Number of points: 6587
Saving data

Considering merra point at [116.875, -33.5]
Index 475, 113
6.736527502400019
Getting ROI
MERRA value: 0.071863495 taken at time 6hrs
Gridding viirs
Counting number of points
Number of points: 7242
Saving data

Considering merra point at [117.5, -33.5]
Index 476, 113
4.674152502400017
Getting ROI
MERRA value: 0.074762665 taken at time 6hrs
Gridding viirs
Counting number of points
Number of points: 5392
Saving data

Considering merra point at [118.125, -33.5]
Index 477, 113
3.3930275024000136
Getting ROI
MERRA value: 0.08327707 taken at time 6hrs
Gridding viirs
Counting number of points
Number of points: 2203
Saving data

Considering merra point at [118.75, -33.5]
Index 478, 113
2.8931525024000107
Getting ROI
MERRA value: 0.09841379 taken at time 6hrs
Gridding viirs
Counting number of points
Number of points: 333
Saving data

Consider

Counting number of points
Number of points: 4954
Saving data

Considering merra point at [-136.25, -33.0]
Index 70, 114
0.33818239999999444
Getting ROI
MERRA value: 0.073853515 taken at time 23hrs
Gridding viirs
Counting number of points
Number of points: 4161
Saving data

Considering merra point at [-135.625, -33.0]
Index 71, 114
0.009807400000000379
Getting ROI
MERRA value: 0.07415869 taken at time 23hrs
Gridding viirs
Counting number of points
Number of points: 1566
Saving data

Considering merra point at [-135.0, -33.0]
Index 72, 114
0.46268240000000627
Getting ROI
MERRA value: 0.083436035 taken at time 23hrs
Gridding viirs
Counting number of points
Number of points: 1637
Saving data

Considering merra point at [-134.375, -33.0]
Index 73, 114
1.6968074000000122
Getting ROI
MERRA value: 0.08068945 taken at time 23hrs
Gridding viirs
Counting number of points
Number of points: 4960
Saving data

Considering merra point at [-133.75, -33.0]
Index 74, 114
3.7121824000000183
Getting ROI
ME

Reading JRR-AOD_v2r1_j01_s202003202121104_e202003202122350_c202003202208470.nc
Gridding viirs
Counting number of points
Number of points: 19
Discarding datapoint

Considering merra point at [-105.0, -33.0]
Index 120, 114
27.094459437200122
Getting ROI
MERRA value: 0.061900992 taken at time 21hrs
Gridding viirs
Counting number of points
Number of points: 21
Discarding datapoint

Considering merra point at [-104.375, -33.0]
Index 121, 114
32.80065943720014
Getting ROI
MERRA value: 0.06429662 taken at time 21hrs
Gridding viirs
Counting number of points
Number of points: 12
Discarding datapoint

Considering merra point at [-103.75, -33.0]
Index 122, 114
39.288109437200156
Getting ROI
MERRA value: 0.06890477 taken at time 21hrs
Gridding viirs
Counting number of points
Number of points: 37
Discarding datapoint

Considering merra point at [-103.125, -33.0]
Index 123, 114
46.55680943720017
Too far from center of VIIRS data.

Considering merra point at [-102.5, -33.0]
Index 124, 114
54.60675943

Reading JRR-AOD_v2r1_j01_s202003201759120_e202003201800348_c202003201846010.nc
Gridding viirs
Counting number of points
Number of points: 7815
Saving data

Considering merra point at [-65.625, -33.0]
Index 183, 114
35.927527888400085
Getting ROI
MERRA value: 0.044954795 taken at time 18hrs
Gridding viirs
Counting number of points
Number of points: 7762
Saving data

Considering merra point at [-65.0, -33.0]
Index 184, 114
28.839027888400075
Getting ROI
MERRA value: 0.04298641 taken at time 18hrs
Gridding viirs
Counting number of points
Number of points: 7654
Saving data

Considering merra point at [-64.375, -33.0]
Index 185, 114
22.531777888400068
Getting ROI
MERRA value: 0.042910118 taken at time 18hrs
Gridding viirs


KeyboardInterrupt: 

In [ ]:
# last index 228 lon, 114 lat